##### データセットの場所やバッチサイズなどの定数値の設定

In [ ]:
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'


# 使用するデバイス
# GPU を使用しない環境（CPU環境）で実行する場合は DEVICE = 'cpu' とする
DEVICE = 'cuda:0'

# 全ての訓練データを一回ずつ使用することを「1エポック」として，何エポック分学習するか
N_EPOCHS = 50

# 学習時のバッチサイズ
BATCH_SIZE = 25

# 訓練データセット（画像ファイルリスト）のファイル名
TRAIN_DATASET_CSV = './tinySTL10/train_list.csv'

# テストデータセット（画像ファイルリスト）のファイル名
TEST_DATASET_CSV = './tinySTL10/test_list.csv'

# 画像ファイルの先頭に付加する文字列（データセットが存在するディレクトリのパス）
DATA_DIR = './tinySTL10'

# 学習結果の保存先フォルダ
MODEL_DIR = './Att_models'

# 学習結果のニューラルネットワークの保存先
MODEL_FILE = os.path.join(MODEL_DIR, 'Att_object_recognizer_model.pth')

##### ニューラルネットワークモデルの定義

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms


# 畳込み，バッチ正規化，ReLUをセットで行うクラス
class myConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
        super(myConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding)
        self.bn = nn.BatchNorm2d(num_features=out_channels)
    def forward(self, x):
        return F.relu(self.bn(self.conv(x)))


# Spatial Attention を実現するクラス
class SpAttBlock(nn.Module):
    def __init__(self):
        super(SpAttBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels=2, out_channels=1, kernel_size=7, stride=1, padding=3)
    def forward(self, x):
        mx, _ = torch.max(x, dim=1, keepdim=True) # チャンネル方向の max pooling
        ax = torch.mean(x, dim=1, keepdim=True) # チャンネル方向の average pooling
        h = torch.cat([mx, ax], dim=1) # 連結
        h = torch.sigmoid(self.conv(h)) # 7x7畳込み + sigmoid
        return h * x


# Channel Attention を実現するクラス
class ChAttBlock(nn.Module):
    def __init__(self, channels):
        super(ChAttBlock, self).__init__()
        self.fc1 = nn.Linear(in_features=channels, out_features=channels)
        self.fc2 = nn.Linear(in_features=channels, out_features=channels)
    def forward(self, x):
        mx, _ = torch.max(x, dim=3) # Global Max Pooling (GMP)
        mx, _ = torch.max(mx, dim=2) # GMP続き
        ax = torch.mean(x, dim=(2, 3)) # Global Average Pooling (GAP)
        mh = self.fc2(F.relu(self.fc1(mx))) # GMPの結果をMLPに通す
        ah = self.fc2(F.relu(self.fc1(ax))) # GMPの結果をMLPに通す
        h = torch.sigmoid(mh + ah) # 加算して sigmoid を適用
        return h.view(*h.size(), 1, 1) * x


# Channel Attention と Spatial Attention をこの順で接続し，さらに shortcut path を追加した Attention Block
class AttBlock(nn.Module):
    def __init__(self, channels):
        super(AttBlock, self).__init__()
        self.ca = ChAttBlock(channels=channels)
        self.sa = SpAttBlock()
    def forward(self, x):
        h = self.ca(x)
        h = self.sa(h)
        return F.relu(x + h)


# STL10物体画像認識AIを実現するニューラルネットワーク（データ拡張およびアテンションなし）
class STL10Recognizer(nn.Module):

    # N: 認識対象となるクラスの数
    def __init__(self, N):
        super(STL10Recognizer, self).__init__()

        # 前処理
        self.preprocess = transforms.Resize(224, antialias=False) # 画像サイズを 224x224 ピクセルにリサイズする（ResNet同じ条件にするため）

        # 畳込み層
        self.conv1 = myConv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = myConv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = myConv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv4 = myConv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv5 = myConv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1)

        # average-pooling層
        self.pool1 = nn.AvgPool2d(kernel_size=2)
        self.pool2 = nn.AvgPool2d(kernel_size=2)
        self.pool3 = nn.AvgPool2d(kernel_size=2)
        self.pool4 = nn.AvgPool2d(kernel_size=2)

        # 平坦化
        self.flat = nn.Flatten()

        # 全結合層
        # 前処理で画像サイズが 224x224 ピクセルになり，その後，pooling層を4回挟むので，特徴マップのサイズは (224/16)x(224/16) = 14x14 ピクセルになる
        self.fc1 = nn.Linear(in_features=128*14*14, out_features=256)
        self.fc2 = nn.Linear(in_features=256, out_features=N)

    def forward(self, x, testmode=False):
        h = self.preprocess(x)
        h = self.conv1(h)
        h = self.pool1(h)
        h = self.conv2(h)
        h = self.pool2(h)
        h = self.conv3(h)
        h = self.pool3(h)
        h = self.conv4(h)
        h = self.pool4(h)
        h = self.conv5(h)
        h = self.flat(h)
        h = F.relu(self.fc1(h))
        y = self.fc2(h)
        return y


# STL10物体画像認識AIを実現するニューラルネットワーク（データ拡張およびアテンションあり）
class STL10RecognizerPlus(nn.Module):

    # N: 認識対象となるクラスの数
    def __init__(self, N):
        super(STL10RecognizerPlus, self).__init__()

        # データ拡張
        self.data_augment = transforms.RandomApply(torch.nn.ModuleList([
            transforms.RandomHorizontalFlip(p=0.5), # 確率0.5で左右反転
            transforms.ColorJitter(brightness=0.2, contrast=0.15, saturation=0.1, hue=0.05), # カラージッター（明度を±20%，コントラストを±15%，彩度を±10%，色相を±5%の範囲内でランダムに変更）
            transforms.RandomAffine(degrees=(-15, 15), scale=(0.8, 1.2)), # -15度〜15度の範囲でランダムに回転，さらに80%〜120%の範囲内でランダムにスケーリング
            transforms.RandomErasing(p=0.5), # 確率0.5で一部を消去
        ]), p=0.5)

        # 前処理
        self.preprocess = transforms.Resize(224, antialias=False) # 画像サイズを 224x224 ピクセルにリサイズする（ResNet同じ条件にするため）

        # 畳込み層
        self.conv1 = myConv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = myConv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = myConv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv4 = myConv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv5 = myConv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1)

        # average-pooling層
        self.pool1 = nn.AvgPool2d(kernel_size=2)
        self.pool2 = nn.AvgPool2d(kernel_size=2)
        self.pool3 = nn.AvgPool2d(kernel_size=2)
        self.pool4 = nn.AvgPool2d(kernel_size=2)

        # Attention Block
        self.att1 = AttBlock(channels=16)
        self.att2 = AttBlock(channels=32)
        self.att3 = AttBlock(channels=64)
        self.att4 = AttBlock(channels=128)

        # 平坦化
        self.flat = nn.Flatten()

        # 全結合層
        # 前処理で画像サイズが 224x224 ピクセルになり，その後，pooling層を4回挟むので，特徴マップのサイズは (224/16)x(224/16) = 14x14 ピクセルになる
        self.fc1 = nn.Linear(in_features=128*14*14, out_features=256)
        self.fc2 = nn.Linear(in_features=256, out_features=N)

    def forward(self, x, testmode=False):
        if not testmode:
            x = self.data_augment(x) # 訓練時のみデータ拡張（テスト時は実行しない）
        h = self.preprocess(x)
        h = self.conv1(h)
        h = self.pool1(h)
        h = self.att1(h) # Attention Mechanism
        h = self.conv2(h)
        h = self.pool2(h)
        h = self.att2(h) # Attention Mechanism
        h = self.conv3(h)
        h = self.pool3(h)
        h = self.att3(h) # Attention Mechanism
        h = self.conv4(h)
        h = self.pool4(h)
        h = self.att4(h) # Attention Mechanism
        h = self.conv5(h)
        h = self.flat(h)
        h = F.relu(self.fc1(h))
        y = self.fc2(h)
        return y

##### 訓練データセットの読み込み

In [ ]:
import pickle
from torch.utils.data import DataLoader, random_split
from mylib.data_io import CSVBasedDataset


# CSVファイルを読み込み, 訓練データセットを用意
dataset = CSVBasedDataset(
    filename = TRAIN_DATASET_CSV,
    items = [
        'File Path', # X
        'Class Name' # Y
    ],
    dtypes = [
        'image', # Xの型
        'label' # Yの型
    ],
    dirname = DATA_DIR,
    img_mode = 'color' # 強制的にカラー画像として読み込む
)
with open(os.path.join(MODEL_DIR, 'fdicts.pkl'), 'wb') as fdicts_file:
    pickle.dump(dataset.forward_dicts, fdicts_file)

# 認識対象のクラス数を取得
n_classes = len(dataset.forward_dicts[1])

# 訓練データセットを分割し，一方を検証用に回す
dataset_size = len(dataset)
valid_size = int(0.1 * dataset_size) # 全体の 10% を検証用に
train_size = dataset_size - valid_size # 残りの 90% を学習用に
train_dataset, valid_dataset = random_split(dataset, [train_size, valid_size])

# 訓練データおよび検証用データをミニバッチに分けて使用するための「データローダ」を用意
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True)

##### 学習処理の実行

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from mylib.visualizers import LossVisualizer


# ニューラルネットワークの作成
model = STL10Recognizer(N=n_classes).to(DEVICE) # データ拡張・Attentionなしバージョンを用いる場合
#model = STL10RecognizerPlus(N=n_classes).to(DEVICE) # データ拡張・Attentionありバージョンを用いる場合

# 最適化アルゴリズムの指定（ここでは SGD でなく Adam を使用）
optimizer = optim.Adam(model.parameters())

# 損失関数：クロスエントロピー損失を使用
loss_func =  nn.CrossEntropyLoss()

# 損失関数値を記録する準備
loss_viz = LossVisualizer(['train loss', 'valid loss'])

# 勾配降下法による繰り返し学習
for epoch in range(N_EPOCHS):

    print('Epoch {0}:'.format(epoch + 1))

    # 学習
    model.train()
    sum_loss = 0
    for X, Y in tqdm(train_dataloader): # X, Y は CSVBasedDataset クラスの __getitem__ 関数の戻り値に対応
        for param in model.parameters():
            param.grad = None
        X = X.to(DEVICE)
        Y = Y.to(DEVICE)
        Y_pred = model(X) # 入力画像 X を現在のニューラルネットワークに入力し，出力の推定値を得る
        loss = loss_func(Y_pred, Y) # 損失関数の現在値を計算
        loss.backward() # 誤差逆伝播法により，個々のパラメータに関する損失関数の勾配（偏微分）を計算
        optimizer.step() # 勾配に沿ってパラメータの値を更新
        sum_loss += float(loss) * len(X)
    avg_loss = sum_loss / train_size
    loss_viz.add_value('train loss', avg_loss) # 訓練データに対する損失関数の値を記録
    print('train loss = {0:.6f}'.format(avg_loss))

    # 検証
    model.eval()
    sum_loss = 0
    n_failed = 0
    with torch.inference_mode():
        for X, Y in tqdm(valid_dataloader):
            X = X.to(DEVICE)
            Y = Y.to(DEVICE)
            Y_pred = model(X, testmode=True)
            loss = loss_func(Y_pred, Y)
            sum_loss += float(loss) * len(X)
            n_failed += torch.count_nonzero(torch.argmax(Y_pred, dim=1) - Y) # 推定値と正解値が一致していないデータの個数を数える
    avg_loss = sum_loss / valid_size
    loss_viz.add_value('valid loss', avg_loss) # 検証用データに対する損失関数の値を記録
    accuracy = (valid_size - n_failed) / valid_size
    print('valid loss = {0:.6f}'.format(avg_loss))
    print('accuracy = {0:.2f}%'.format(100 * accuracy))
    print('')

# 学習結果のニューラルネットワークモデルをファイルに保存
model = model.to('cpu')
torch.save(model.state_dict(), MODEL_FILE)

# 損失関数の記録をファイルに保存
loss_viz.save(v_file=os.path.join(MODEL_DIR, 'loss_graph.png'), h_file=os.path.join(MODEL_DIR, 'loss_history.csv'))

##### テストデータセットの読み込み

In [ ]:
import pickle
from torch.utils.data import DataLoader
from mylib.data_io import CSVBasedDataset


# CSVファイルを読み込み, テストデータセットを用意
with open(os.path.join(MODEL_DIR, 'fdicts.pkl'), 'rb') as fdicts_file:
    fdicts = pickle.load(fdicts_file)
test_dataset = CSVBasedDataset(
    filename = TEST_DATASET_CSV,
    items = [
        'File Path', # X
        'Class Name' # Y
    ],
    dtypes = [
        'image', # Xの型
        'label' # Yの型
    ],
    dirname = DATA_DIR,
    fdicts = fdicts,
    img_mode = 'color' # 強制的にカラー画像として読み込む
)
test_size = len(test_dataset)
rdict = test_dataset.reverse_dicts[1]

# 認識対象のクラス数を取得
n_classes = len(test_dataset.forward_dicts[1])

# テストデータをミニバッチに分けて使用するための「データローダ」を用意
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True)

##### 学習済みニューラルネットワークモデルのロード

In [ ]:
import torch


# ニューラルネットワークモデルとその学習済みパラメータをファイルからロード
model = STL10Recognizer(N=n_classes) # データ拡張・Attentionなしバージョンを用いる場合
#model = STL10RecognizerPlus(N=n_classes) # データ拡張・Attentionありバージョンを用いる場合
model.load_state_dict(torch.load(MODEL_FILE))

##### 単一画像に対するテスト処理の実行

In [ ]:
import torch
from mylib.data_io import show_single_image


model = model.to(DEVICE)
model.eval()

# index 番目のテストデータをニューラルネットワークに入力してみる
while True:
    print('index?: ', end='')
    val = input()
    if val == 'exit': # 'exit' とタイプされたら終了
        break
    index = int(val)
    x, y = test_dataset[index]
    x = x.reshape(1, *x.size()).to(DEVICE)
    with torch.inference_mode():
        y_pred = model(x, testmode=True)
    y_pred = torch.argmax(y_pred, dim=1)
    print('')
    print('estimated:', rdict[int(y_pred)])
    print('ground truth:', rdict[int(y)])
    print('')
    show_single_image(x.to('cpu'), title='input image', sec=1)

##### 全ての画像に対するテスト処理の実行

In [ ]:
import torch
from tqdm import tqdm


model = model.to(DEVICE)
model.eval()

# テストデータセットを用いて認識精度を評価
n_failed = 0
with torch.inference_mode():
    for X, Y in tqdm(test_dataloader):
        X = X.to(DEVICE)
        Y = Y.to(DEVICE)
        Y_pred = model(X, testmode=True)
        n_failed += torch.count_nonzero(torch.argmax(Y_pred, dim=1) - Y) # 推定値と正解値が一致していないデータの個数を数える
    accuracy = (test_size - n_failed) / test_size
    print('accuracy = {0:.2f}%'.format(100 * accuracy))
    print('')